In [ ]:
!pip install -q --upgrade transformers
!pip install -q einops xmltodict
!pip install -q bitsandbytes trl peft accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 45.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jul 25 22:07:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   42C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
import xmltodict
import sys

from tqdm import tqdm


def replace_marks(text: str) -> str:
    text = text.replace('&amp;', '&')
    text = text.replace('&quot;', '"')
    text = text.replace('&apost;', "'")
    return text

def read_cnn_corpus(corpus_dir: str) -> dict:
    assert os.path.exists(corpus_dir)
    documents_names = os.listdir(corpus_dir)
    corpus = []
    with tqdm(total=len(documents_names), file=sys.stdout, colour='blue',
              desc='Reading Corpus') as pbar:
      for doc_name in documents_names:
          document_file = os.path.join(corpus_dir, doc_name)
          with open(document_file, encoding='utf-8') as file:
              xml_doc = xmltodict.parse(file.read())
          xml_doc = xml_doc['document']
          title = xml_doc['title']
          title = replace_marks(title)
          highlights_element = xml_doc['summaries']['highlights']
          highlights = ''
          for sentence_highlight in highlights_element['sentence']:
              highlights += sentence_highlight['#text'] + '\n'
          highlights = highlights.strip()
          highlights = replace_marks(highlights)
          gold_standard_element = xml_doc['summaries']['gold_standard']
          gold_standard = ''
          if isinstance(gold_standard_element['sentence'], dict):
              gold_standard += gold_standard_element['sentence']['#text']
          else:
              for sentence_gold_standard in gold_standard_element['sentence']:
                  if isinstance(sentence_gold_standard, dict):
                      gold_standard += sentence_gold_standard['#text'] + '\n'
          gold_standard = gold_standard.strip()
          gold_standard = replace_marks(gold_standard)
          article_element = xml_doc['article']
          text = ''
          for paragraph_element in article_element['paragraph']:
              sentences_element = paragraph_element['sentences']['sentence']
              if isinstance(sentences_element, list):
                  for sentence_element in sentences_element:
                      text += sentence_element['content'] + ' '
              else:
                  text += sentences_element['content'] + ' '
          text = text.strip()
          text = replace_marks(text)
          doc_name = doc_name.replace('.xml', '').replace("'", '').lower()
          name = doc_name.replace(';', '').replace('&', '').replace('%', '').strip()
          document = {
              'name': name,
              'title': title,
              'highlights': highlights,
              'gold_standard': gold_standard,
              'text': text
          }
          corpus.append(document)
          pbar.update(1)
    return corpus


def save_summary(document_name: str, summary: str, summaries_dir: str,
                 summary_name: str):
    document_dir = os.path.join(summaries_dir, document_name.lower())
    os.makedirs(document_dir, exist_ok=True)
    summary_path = os.path.join(document_dir, f'{summary_name}.txt')
    with open(summary_path, 'w', encoding='utf-8') as file:
        file.write(summary)

In [ ]:
## Montar Drive AQUI

In [ ]:
corpus_dir = f'/content/drive/My Drive/Experimentos/abs_summ_benchmark/corpora/cnn'

summaries_dir = f'/content/drive/My Drive/Experimentos/abs_summ_benchmark/summaries/cnn'

os.makedirs(summaries_dir, exist_ok=True)

corpus_cnn = read_cnn_corpus(corpus_dir)

Reading Corpus: 100%|██████████| 3000/3000 [01:32<00:00, 32.55it/s] 


In [ ]:
print(f'Total of Documents: {len(corpus_cnn)}')

print('\nText:', corpus_cnn[0]['text'])

Total of Documents: 3000

Text: Amid the swirl of claims and counter-claims involving sex, lies, and witness videotapes, one of the most intriguing aspects of Bo Xilai's so-called "trial of the century" in China has been the live micro-blogging of proceedings from the courthouse. Case updates, photos, excerpts of video testimony and partial transcripts were posted on the court's Twitter-like Sina Weibo account during the course of the five-day trial of the former high-flying politician on charges of bribery, embezzlement and abuse of power, attracting more than 590,000 followers. State media rushed to herald the micro-blog as a reflection of "historic transparency" and "historic progress for the rule of law" in a country that regularly clamps down on the free flow of information and dissent. But while the court's micro-blog offered unprecedented glimpses into what was widely viewed as a tightly-orchestrated show trial, the flow of information was nonetheless a selective drip-feed that 

In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

# model_name = 'llama3_8b_it'
model_name = 'llama31_8b_it'

if model_name == 'llama3_8b_it':
  model_checkpoint = 'meta-llama/Meta-Llama-3-8B-Instruct'
elif model_name == 'llama31_8b_it':
  model_checkpoint = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

print(f'\nModel Name: {model_name} -- {model_checkpoint}')


Model Name: llama31_8b_it -- meta-llama/Meta-Llama-3.1-8B-Instruct


In [ ]:
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_compute_dtype=torch.bfloat16,)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint, quantization_config=quantization_config,
    device_map='auto')

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
template = """
ARTICLE: ```{text}```. Summarize the article in {num_sentences} SENTENCES.
SUMMARY:
"""

print(f'Template: {template}')

Template: 
ARTICLE: ```{text}```. Summarize the article in {num_sentences} SENTENCES.
SUMMARY:



In [ ]:
import warnings

warnings.simplefilter('ignore')

with tqdm(total=len(corpus_cnn), file=sys.stdout,
          colour='green', desc='Summarizing documents') as pbar:

  for document in corpus_cnn:

    num_sentences = len(document['highlights'].split('\n'))

    input_text = document['text']

    document_dir = os.path.join(summaries_dir, document['name'].lower())
    summary_path = os.path.join(document_dir, f'{model_name}.txt')

    if os.path.exists(summary_path):
      pbar.update(1)
      continue

    prompt = template.replace('{text}', input_text)
    prompt = prompt.replace('{num_sentences}', str(num_sentences))

    messages = [
         {
             'role': 'system',
             'content': 'You are a professional summarizer who create news article summaries.'
         },
        {
            'role': 'user',
            'content': prompt
        }
    ]

    input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors='pt'
    ).to(model.device)

    terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids('<|eot_id|>')
    ]

    torch.backends.cuda.enable_mem_efficient_sdp(False)
    torch.backends.cuda.enable_flash_sdp(False)

    outputs = model.generate(
      input_ids,
      max_new_tokens=150,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.3,
      top_p=0.9
    )

    summary_logits = outputs[0][input_ids.shape[-1]:]

    summary = tokenizer.decode(summary_logits, skip_special_tokens=True)

    save_summary(document['name'], summary, summaries_dir, model_name)

    pbar.update(1)

Summarizing documents:  15%|█▍        | 446/3000 [02:50<14:02,  3.03it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Summarizing documents:  15%|█▍        | 447/3000 [03:10<4:18:33,  6.08s/it]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Summarizing documents:  15%|█▍        | 448/3000 [03:25<6:13:35,  8.78s/it]

...

Summarizing documents: 100%|█████████▉| 2998/3000 [8:40:45<00:23, 11.76s/it]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Summarizing documents: 100%|█████████▉| 2999/3000 [8:40:57<00:11, 11.72s/it]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Summarizing documents: 100%|██████████| 3000/3000 [8:41:07<00:00, 10.42s/it]
